# Déchiffrement de la syntaxe Prolog

In [1]:
type var = Id of string * int

type table = Empty | NonEmpty of term * table | TVar of var
and term = Var of var | Predicate of string * (term list) | Table of table

type clause = Clause of term * (term list);;

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



type var = Id of string * int


type table = Empty | NonEmpty of term * table | TVar of var
and term = Var of var | Predicate of string * term list | Table of table


type clause = Clause of term * term list


## Le nouveau Parser :

In [2]:
type structure = S of string | V of var | L of table | T of term | C of clause 
                | W of clause list | P of string | TL of term list 


type structure =
    S of string
  | V of var
  | L of table
  | T of term
  | C of clause
  | W of clause list
  | P of string
  | TL of term list


In [3]:
let to_list str = List.init (String.length str) (String.get str);;
let rec remove_spaces = function
| [] -> []
| (' ' | '\t' | '\n')::l -> remove_spaces l
| a::l -> a::(remove_spaces l);;
let (|>) a b = b a;;
let of_char c = String.make 1 c;;

val to_list : string -> char list = <fun>


val remove_spaces : char list -> char list = <fun>


val ( |> ) : 'a -> ('a -> 'b) -> 'b = <fun>


val of_char : char -> string = <fun>


In [4]:
let ( <+> ) ra rb = fun l -> (ra l) @ (rb l)

let chain y l = List.map (fun (a, b) -> (a, y@b)) l

let ( *~* ) ra rb = 
let chain y l = List.map (fun (a, b) -> (a, y@b)) l in
                    fun l -> List.concat (
                        List.map (fun (x, y) -> chain y (rb x)) (ra l)
                    )
                    
let ( >~> ) ra f = fun l -> List.map (fun (a, b) -> (a, f b)) (ra l)

let skip = fun l -> []
                    
let fix aux = let rec g x = aux g x in g

val ( <+> ) : ('a -> 'b list) -> ('a -> 'b list) -> 'a -> 'b list = <fun>


val chain : 'a list -> ('b * 'a list) list -> ('b * 'a list) list = <fun>


val ( *~* ) :
  ('a -> ('b * 'c list) list) ->
  ('b -> ('d * 'c list) list) -> 'a -> ('d * 'c list) list = <fun>


val ( >~> ) : ('a -> ('b * 'c) list) -> ('c -> 'd) -> 'a -> ('b * 'd) list =
  <fun>


val skip : 'a -> 'b list = <fun>


val fix : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


In [5]:
let rlow = function
| ('a'..'z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rup = function
| ('A'..'Z' as c)::l -> [l, [S (of_char c)]]
| _ -> []

let rsym s = function
| h::t when s = h -> [t, [S (of_char s)]]
| _ -> []

let rchar = function
| ('a'..'z' | 'A'..'Z' | '0'..'9' | '_' as c)::t -> [t, [S (of_char c)]]
| _ -> []

let rmot =
let aux self = rchar 
            <+> rchar *~* self >~> (function | [S s; S s'] -> [S (s^s')] | l -> l)
in fix aux

let variable = 
    rup >~> (function | [S s] -> [V (Id (s,0) )] | l -> l)
<+> rup *~* rmot >~> (function | [S s; S s'] -> [V (Id ((s^s'),0) )] | l -> l)

let predicate = rlow >~> (function | [S s] -> [P s] | l -> l)
                <+> rlow *~* rmot >~> (function | [S s; S s'] -> [P (s^s')] | l -> l)

let rec table_from_list tl t = match tl with
| h::q -> NonEmpty (h, table_from_list q t)
| [] -> t

let rec term l = (
    variable >~> (function | [V s] -> [T (Var s)] | l -> l)
<+> predicate >~> (function | [P s] -> [T (Predicate (s,[]))] | l -> l)
<+> predicate *~* (rsym '(' >~> skip) *~* termlist *~* (rsym ')' >~> skip) 
        >~> (function | [P s; TL tl] -> [T (Predicate (s, tl))] | l -> l)
<+> table >~> (function | [L t] -> [T (Table t)] | l -> l)
) l
and termlist l = (
term >~> (function | [T t] -> [TL [t]] | l -> l)
<+> term *~* (rsym ',' >~> skip) *~* termlist >~> (function | [T t; TL tl] -> [TL (t::tl)] | l -> l)
) l
and table l = (
rsym '[' *~* rsym ']' >~> (function | [S "["; S "]"] -> [L Empty] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl] -> [L (table_from_list tl Empty)] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym '|' >~> skip) *~* table *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl; L t] -> [L (table_from_list tl t)] | l -> l)
<+> (rsym '[' >~> skip) *~* termlist *~* (rsym '|' >~> skip) *~* variable *~* (rsym ']' >~> skip) 
    >~> (function | [TL tl; V v] -> [L (table_from_list tl (TVar v))] | l -> l)
) l


let clause = term *~* (rsym '.' >~> skip) >~> (function | [T t] -> [C (Clause (t,[]))] | l -> l)
            <+> term *~* (rsym ':' *~* rsym '-' >~> skip) *~* termlist *~* (rsym '.' >~> skip)
                    >~> (function | [T t; TL tl] -> [C (Clause (t,tl))] | l -> l)
let programme =
let aux self = clause >~> (function | [C c] -> [W [c]] | l -> l)
            <+> clause *~* self >~> (function | [C c; W cl] -> [W (c::cl)]| l -> l)
in fix aux

let check r l = 
let aux = function
| [], result -> Some result
| _ -> None
in match List.filter_map aux (r l) with
| [] -> failwith "Parsing failed"
| [a] -> a
| _ -> failwith "Ambiguous grammar";;

let parse_program s = match (s |> to_list |> remove_spaces |> (check programme)) with
| [W w] -> w
| _ -> failwith "input is not a program"

let parse_term s = match (s |> to_list |> remove_spaces |> (check term)) with
| [T t] -> t
| _ -> failwith "input is not a term"

let parse_clause c = match (c |> to_list |> remove_spaces |> (check clause)) with
| [C a] -> a
| _ -> failwith "input is not a clause"

let parse_termlist str = match (str |> to_list |> remove_spaces |> (check termlist)) with
| [TL tl] -> tl
| _ -> failwith "input is not a termlist"

val rlow : char list -> (char list * structure list) list = <fun>


val rup : char list -> (char list * structure list) list = <fun>


val rsym : char -> char list -> (char list * structure list) list = <fun>


val rchar : char list -> (char list * structure list) list = <fun>


val rmot : char list -> (char list * structure list) list = <fun>


...

val check : ('a -> ('b list * 'c) list) -> 'a -> 'c = <fun>


val parse_program : string -> clause list = <fun>


val parse_term : string -> term = <fun>


val parse_clause : string -> clause = <fun>


val parse_termlist : string -> term list = <fun>


In [6]:
let chaine =  "apprend([a, b, c | [d, e, f | A]]).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).
etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
";;
parse_program chaine;;

val chaine : string =
  "apprend([a, b, c | [d, e, f | A]]).\napprend(benjamin, informatique).\napprend(benjamin, physique).\nenseigne(alice, physique).\nenseigne(pierre, mathematiques).\nenseigne(pierre, informatique).\netudiant_de(E,P):-apprend(E,M), enseigne(P,M).\n"


- : clause list =
[Clause
  (Predicate ("apprend",
    [Table
      (NonEmpty (Predicate ("a", []),
        NonEmpty (Predicate ("b", []),
         NonEmpty (Predicate ("c", []),
          NonEmpty (Predicate ("d", []),
           NonEmpty (Predicate ("e", []),
            NonEmpty (Predicate ("f", []), TVar (Id ("A", 0)))))))))]),
  []);
 Clause
  (Predicate ("apprend",
    [Predicate ("benjamin", []); Predicate ("informatique", [])]),
  []);
 Clause
  (Predicate ("apprend",
    [Predicate ("benjamin", []); Predicate ("physique", [])]),
  []);
 Clause
  (Predicate ("enseigne",
    [Predicate ("alice", []); Predicate ("physique", [])]),
  []);
 Clause
  (Predicate ("enseigne",
    [Predicate ("pierre", []); Predicate ("mathematiques", [])]),
  []);
 Clause
  (Predicate ("enseigne",
    [Predicate ("pierre", []); Predicate ("informatique", [])]),
  []);
 Clause (Predicate ("etudiant_de", [Var (Id ("E", 0)); Var (Id ("P", 0))]),
  [Predicate ("apprend", [Var (Id ("E", 0)); Var (Id ("M", 

# Fonctions de base

In [7]:
let rec string_of_term = function
| Predicate (p, []) -> p
| Predicate (p, l) -> p ^ "(" ^ (String.concat ", " (List.map string_of_term l)) ^ ")"
| Var (Id (s, _)) -> s
| Table (Empty) -> "[]"
| Table (TVar (Id (s,_))) -> s
| Table (NonEmpty (t, l)) -> 
let rec aux = function
| Empty -> ""
| TVar (Id (s, _)) -> "|"^s
| NonEmpty (t, l) -> ", "^(string_of_term t)^(aux l)
in "["^(string_of_term t)^(aux l)^"]"

let print_eql eql = 
List.iter (fun (t1,t2) -> Format.printf "%s <-> %s\n%!" (string_of_term t1) (string_of_term t2)) eql

val string_of_term : term -> string = <fun>


val print_eql : (term * term) list -> unit = <fun>


In [8]:
(* Teste si un terme est dans un terme *)
let rec term_in_term (tv : term) = function
| (Var _ | Table Empty | Table (TVar _) as term) -> tv = term
| Predicate (_, l) as term -> tv = term || List.mem true (List.map (term_in_term tv) l)
| Table (NonEmpty (head, tail)) as term -> tv = term || term_in_term tv head || term_in_term tv (Table tail);;

term_in_term (Var (Id ("P", 0))) (parse_term "[A,[B|Z],C | [E | P] ]");;

(* Remplace un terme par un nouveau terme, récursivement à l'intérieur d'un terme *)
let rec replace_term_in_term old_term new_term = function
| term when term = old_term -> new_term
| Predicate (a , l) -> Predicate (a, List.map (replace_term_in_term old_term new_term) l)
| Table (NonEmpty (head, tail)) -> (
    match replace_term_in_term old_term new_term (Table tail) with
    | Table n -> Table (NonEmpty (replace_term_in_term old_term new_term head, n))
    | _ -> failwith "You tried to replace a subtable by a term that is not a table !" )
| term -> term;;

replace_term_in_term (parse_term "E") (parse_term "pierre") (parse_term "etudiant_de([E,B | E],P)");;
replace_term_in_term (Table (TVar (Id ("E", 0)))) (parse_term "[pierre]") (parse_term "etudiant_de([E,B | E],P)");;

(* Teste si un terme est dans une liste de couples de termes *)
let term_in_eql term eql = List.mem true (List.map 
(function t -> term_in_term term (fst t) || term_in_term term (snd t))
eql);;

(* Remplace une variable par un nouveau terme, à l'intérieur d'une liste de couples de termes *)
let replace_term_in_eql old_term new_term eql = 
List.map (function (a, b) -> replace_term_in_term old_term new_term a, replace_term_in_term old_term new_term b) eql;;

let rec table_in_term table = function
| Var v -> false
| Predicate (p, l) -> List.mem true (List.map (table_in_term table) l)
| Table t -> table = t || (match t with 
| Empty | TVar _ -> false
| NonEmpty (head, tail) -> table_in_term table head || table_in_term table (Table tail))


let rec replace_table_in_term old_table new_table = function
| Var v -> Var v
| Predicate (p, l) -> Predicate (p, List.map (replace_table_in_term old_table new_table) l)
| Table t -> if old_table = t then Table new_table else Table (
match t with
| Empty -> Empty
| TVar v -> TVar v
| NonEmpty (head, tail) -> (match replace_table_in_term old_table new_table (Table tail) with
    | Table n -> NonEmpty (replace_table_in_term old_table new_table head, n)
    | Var _ | Predicate (_, _) -> failwith "I don't know"
    )
)

(* Teste si une table est dans une liste de couples de termes *)
let table_in_eql table eql = List.mem true (List.map 
(function t -> table_in_term table (fst t) || table_in_term table (snd t))
eql);;

(* Remplace une variable par un nouveau terme, à l'intérieur d'une liste de couples de termes *)
let replace_table_in_eql old_table new_table eql = 
List.map (function (a, b) -> 
(replace_table_in_term old_table new_table a), (replace_table_in_term old_table new_table a)) eql;;

val term_in_term : term -> term -> bool = <fun>


- : bool = false


val replace_term_in_term : term -> term -> term -> term = <fun>


- : term =
Predicate ("etudiant_de",
 [Table
   (NonEmpty (Predicate ("pierre", []),
     NonEmpty (Var (Id ("B", 0)), TVar (Id ("E", 0)))));
  Var (Id ("P", 0))])


- : term =
Predicate ("etudiant_de",
 [Table
   (NonEmpty (Var (Id ("E", 0)),
     NonEmpty (Var (Id ("B", 0)), NonEmpty (Predicate ("pierre", []), Empty))));
  Var (Id ("P", 0))])


...

val replace_term_in_eql :
  term -> term -> (term * term) list -> (term * term) list = <fun>


val table_in_term : table -> term -> bool = <fun>


val replace_table_in_term : table -> table -> term -> term = <fun>


val table_in_eql : table -> (term * term) list -> bool = <fun>


val replace_table_in_eql :
  table -> table -> (term * 'a) list -> (term * term) list = <fun>


# Vérification des types

In [9]:
let rec list_tvars_in_term = function
| Var _ -> []
| Predicate (_, l) -> List.concat (List.map list_tvars_in_term l)
| Table t -> 
let rec aux = function
| Empty -> []
| NonEmpty (head, tail) -> (list_tvars_in_term head) @ (aux tail)
| TVar v -> [v]
in aux t

let rec list_tvars_in_termlist tl = List.concat (List.map list_tvars_in_term tl)

let list_tvars_in_clause (Clause (t, tl)) = (list_tvars_in_term t) @ (list_tvars_in_termlist tl)

let type_check_cl cl = 
let l = list_tvars_in_clause cl in
List.fold_left (fun cl v -> 
let aux = replace_term_in_term (Var v) (Table (TVar v))
and Clause (left, right) = cl in
Clause (aux left, List.map aux right)
) cl l

let type_check_term term =
List.fold_left (fun t v -> replace_term_in_term (Var v) (Table (TVar v)) t) term (list_tvars_in_term term)

let read_term str = type_check_term (parse_term str)
let read_program str = List.map type_check_cl (parse_program str)
let read_clause str = type_check_cl (parse_clause str)
let read_termlist str = List.map type_check_term (parse_termlist str)

val list_tvars_in_term : term -> var list = <fun>


val list_tvars_in_termlist : term list -> var list = <fun>


val list_tvars_in_clause : clause -> var list = <fun>


val type_check_cl : clause -> clause = <fun>


val type_check_term : term -> term = <fun>


val read_term : string -> term = <fun>


val read_program : string -> clause list = <fun>


val read_clause : string -> clause = <fun>


val read_termlist : string -> term list = <fun>


# Algorithme d'unification

In [10]:
(* E est une liste de couples de termes *)
(* transforme E en un équivalent sous forme résolue (type option, None si impossible) *)
let rec solve e =
    (* Parcourt une fois l'ensemble des équations *)
    let rec pass tmp = function
    | [] -> Some tmp
    
    | (Predicate (f, lf), Predicate (g, lg))::l when f = g && List.length lf = List.length lg ->
        pass ((List.map2 (fun a b -> a,b) lf lg)@tmp) l
        
    | (Predicate (_, _), Predicate (_, _))::l -> 
        None (* failwith "Unification failed" *)
    
    | (Var x, Var y)::l when x = y -> 
        pass tmp l
    
    | (Predicate (a, la), Var x)::l -> 
        pass ((Var x, Predicate (a, la))::tmp) l
    
    | (Table t, Var x)::l ->
        pass ((Var x, Table t)::tmp) l
    
    | (Var _ as v, t)::l when term_in_eql v tmp || term_in_eql v l ->
        if term_in_term v t then None (* failwith "Unification failed, loop" *)
        else pass ((v, t)::(replace_term_in_eql v t tmp)) (replace_term_in_eql v t l)
        
    | (Var _ as v, t)::l -> 
        pass ((v, t)::tmp) l
    
    | (Predicate (_, _), Table _)::_ -> None
    
    | (Table _, Predicate (_, _))::_ -> None
    
    | (Table t1, Table t2)::l -> (
        match t1, t2 with 
        | Empty, Empty -> pass tmp l
        
        | Empty, NonEmpty _ -> None
        
        | NonEmpty _, Empty -> None
        
        | NonEmpty (head1, tail1), NonEmpty (head2, tail2) -> pass ((head1, head2)::(Table tail1, Table tail2)::tmp) l
        
        | (NonEmpty (_, _) | Empty as t), TVar a -> pass ((Table (TVar a), Table t)::tmp) l
        
        | TVar a, TVar b when a=b -> pass tmp l
        
        | TVar _ as v, table when table_in_eql v tmp || table_in_eql v l ->
            if table_in_term v (Table table) then None (* Loop *)
            else pass ((Table v, Table table)::(replace_table_in_eql v table tmp)) (replace_table_in_eql v table l)
        
        | TVar _ as v, table -> pass ((Table v, Table table)::tmp) l
        )

    
in match (pass [] e) with 
| None -> None
| Some eqlpp -> 
let b = List.sort_uniq compare eqlpp in if e = b then Some e else solve b;;

val solve : (term * term) list -> (term * term) list option = <fun>


In [11]:
(* Prend 2 terme et renvoie l'unifieur le plus général s'il existe, None sinon. 
Attention, aucune variable ne doit être présente dans (r) la requête et dans (c) la tête de la clause *)

let rec mgu r c = let f = function
    | Var x, t -> x, t
    | Predicate (_, _), _ -> failwith "Should not happen."
    | Table (TVar r), t -> r, t
    | Table (Empty | NonEmpty _), _ -> failwith "Should not happen."
in match solve [r, c] with
| None -> None
| Some e -> Some (List.map f e);;

val mgu : term -> term -> (var * term) list option = <fun>


In [12]:
(* La fonction test_unification a pour but de tester mgu *)
let test_unification str_r str_c = Format.printf "Unification de %s et de %s :\n%!" str_r str_c;
match mgu (str_r |> read_term) (str_c |> read_term) with
| Some t -> List.iter (fun (Id (s, a), term) -> Format.printf "%s <- %s\n%!" s (string_of_term term) ) t
| None -> Format.printf "No unification\n%!"
;;

val test_unification : string -> string -> unit = <fun>


In [13]:
test_unification "etudiant_de(E, pierre)" "etudiant_de(F,P)"; (* F/E et P/pierre *)

test_unification "etudiant_de(F,P)" "etudiant_de(E, pierre)"; (* E/F et P/pierre *)

test_unification "f(X,g(Y))" "f(g(Z),Z)"; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "f(g(Z),Z)" "f(X,g(Y))"; (* X/g(g(Y)) et Z/g(Y) *)

test_unification "[a, a, b, c]" "[A | B]";

test_unification "[a, a, b, c]" "[A | A]"

Unification de etudiant_de(E, pierre) et de etudiant_de(F,P) :
E <- F
P <- pierre
Unification de etudiant_de(F,P) et de etudiant_de(E, pierre) :
F <- E
P <- pierre
Unification de f(X,g(Y)) et de f(g(Z),Z) :
X <- g(g(Y))
Z <- g(Y)
Unification de f(g(Z),Z) et de f(X,g(Y)) :
X <- g(g(Y))
Z <- g(Y)
Unification de [a, a, b, c] et de [A | B] :


- : unit = ()


# Algorithme de backtracking

In [14]:
(* Applique une substitution sur un terme *)
let rec apply_subst_on_term uni term = match uni with
| [] -> term
| (v,t)::l -> 
(match t with
| Table tbl -> apply_subst_on_term l (replace_table_in_term (TVar v) tbl term)
| t -> apply_subst_on_term l (replace_term_in_term (Var v) t term)
)

(* Applique une substitution sur une liste de termes *)
let rec apply_subst_on_termlist uni lst = List.map (apply_subst_on_term uni) lst;;

A <- a
B <- [a, b, c]
Unification de [a, a, b, c] et de [A | A] :
No unification


val apply_subst_on_term : (var * term) list -> term -> term = <fun>


val apply_subst_on_termlist : (var * term) list -> term list -> term list =
  <fun>


In [15]:
(* Prend un entier et une clause, renvoie la clause avec les variables renommées à l'entier *)
let rename n (Clause (t1, tl)) = 
let rec f = function 
| Var (Id (str, _)) -> Var (Id (str, n))
| Predicate (atm, l) -> Predicate (atm, List.map f l)
| Table t -> 
let rec aux = function
| Empty -> Empty
| TVar (Id (str, _)) -> TVar (Id (str, n))
| NonEmpty (head, tail) -> NonEmpty (f head, aux tail)
in Table (aux t)
in Clause (f t1, List.map f tl);;

rename 42 (read_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M).");;

val rename : int -> clause -> clause = <fun>


- : clause =
Clause (Predicate ("etudiant_de", [Var (Id ("E", 42)); Var (Id ("P", 42))]),
 [Predicate ("apprend", [Var (Id ("E", 42)); Var (Id ("M", 42))]);
  Predicate ("enseigne", [Var (Id ("P", 42)); Var (Id ("M", 42))])])


In [16]:
type 'a tree = Leaf of 'a | Node of (('a tree) Lazy.t) list;;

(* La fonction de recherche, renvoie l'arbre des solutions *)
let rec sld_tree world req subs n = match req with
| [] -> Leaf subs 
| head_request_term::other_request_terms -> 
    Node (List.filter_map (fun c -> let Clause (left_member, right_member) = rename n c in 
    (match mgu head_request_term left_member with
        | None -> None
        | Some unifier -> Some (lazy (sld_tree world 
            (apply_subst_on_termlist unifier (right_member@other_request_terms)) 
            (unifier::subs) (n+1)))
    )) world);;

type 'a tree = Leaf of 'a | Node of 'a tree Lazy.t list


val sld_tree :
  clause list ->
  term list -> (var * term) list list -> int -> (var * term) list list tree =
  <fun>


In [17]:
(* Compose correctement la liste de substitutions *)
let compose l = 
let compose2 s1 s2 = s1@(List.map (function (v2, lt2) -> (v2, apply_subst_on_term s1 lt2)) s2)
in List.fold_left compose2 [] l;;

(* Recherche les termes qui sont des variables récursivement dans une liste de termes *)
let rec find_vars_in_termlist tl = 
let rec find_vars_in_term = function
| Var v -> [Var v]
| Predicate (_, l) -> find_vars_in_termlist l
| Table t ->
let rec aux = function
| Empty -> []
| TVar v -> [Table (TVar v)]
| NonEmpty (head, tail) -> (find_vars_in_term head)@(aux tail)
in aux t
in List.sort_uniq compare (List.concat (List.map find_vars_in_term tl));;

val compose : (var * term) list list -> (var * term) list = <fun>


val find_vars_in_termlist : term list -> term list = <fun>


In [18]:
let list_to_seq l = List.fold_right (fun x s -> (fun ()->Seq.Cons(x, s)) ) l Seq.empty;;

let rec to_seq = function
| Leaf str -> Seq.return str
| Node tl -> Seq.flat_map (fun par -> to_seq (Lazy.force par)) (list_to_seq tl);;

let solutions world req = let tree = sld_tree world (read_termlist req) [] 1 in 
let vars = (find_vars_in_termlist (read_termlist req)) in
Seq.map  (fun l -> vars, apply_subst_on_termlist (compose l) vars) (to_seq tree);;

let request world req = let sol = solutions world req in
if sol () = Seq.Nil then Format.printf "This is false.\n%!"
else Seq.iter (fun (vars, tl) -> 
if vars = [] then Format.printf "This is True.\n%!"
else Format.printf "There is : %s\n%!"
(String.concat ", " (List.map2 (fun v t -> (string_of_term v) ^ " = " ^ (string_of_term t)) vars tl))
) sol;;

val list_to_seq : 'a list -> 'a Seq.t = <fun>


val to_seq : 'a tree -> 'a Seq.t = <fun>


val solutions : clause list -> string -> (term list * term list) Seq.t =
  <fun>


val request : clause list -> string -> unit = <fun>


# Essais de l'implémentation

## Tests basiques... des élèves, des cours et des professeurs

In [46]:
let world1 = read_program "apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).
etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
";;

val world1 : clause list =
  [Clause
    (Predicate ("apprend",
      [Predicate ("eve", []); Predicate ("mathematiques", [])]),
    []);
   Clause
    (Predicate ("apprend",
      [Predicate ("benjamin", []); Predicate ("informatique", [])]),
    []);
   Clause
    (Predicate ("apprend",
      [Predicate ("benjamin", []); Predicate ("physique", [])]),
    []);
   Clause
    (Predicate ("enseigne",
      [Predicate ("alice", []); Predicate ("physique", [])]),
    []);
   Clause
    (Predicate ("enseigne",
      [Predicate ("pierre", []); Predicate ("mathematiques", [])]),
    []);
   Clause
    (Predicate ("enseigne",
      [Predicate ("pierre", []); Predicate ("informatique", [])]),
    []);
   Clause (Predicate ("etudiant_de", [Var (Id ("E", 0)); Var (Id ("P", 0))]),
    [Predicate ("apprend", [Var (Id ("E", 0)); Var (Id ("M", 0))]);
     Predicate ("enseigne", [Var (Id ("P", 0)); Var (Id ("M", 0))])])]


In [47]:
read_term "apprend(benjamin, informatique)";;

read_termlist "apprend(E,M), enseigne(P,M)";;

read_clause "etudiant_de(E,P):-apprend(E,M), enseigne(P,M).";;

- : term =
Predicate ("apprend",
 [Predicate ("benjamin", []); Predicate ("informatique", [])])


- : term list =
[Predicate ("apprend", [Var (Id ("E", 0)); Var (Id ("M", 0))]);
 Predicate ("enseigne", [Var (Id ("P", 0)); Var (Id ("M", 0))])]


- : clause =
Clause (Predicate ("etudiant_de", [Var (Id ("E", 0)); Var (Id ("P", 0))]),
 [Predicate ("apprend", [Var (Id ("E", 0)); Var (Id ("M", 0))]);
  Predicate ("enseigne", [Var (Id ("P", 0)); Var (Id ("M", 0))])])


In [48]:
request world1 "etudiant_de(E, pierre)";;

There is : E = eve
There is : E = benjamin


- : unit = ()


In [49]:
request world1 "etudiant_de(E, pierre), etudiant_de(E, alice)";;

There is : E = benjamin


- : unit = ()


In [50]:
request world1 "etudiant_de(A, B)";;

There is : A = eve, B = pierre
There is : A = benjamin, B = pierre
There is : A = benjamin, B = alice


- : unit = ()


In [51]:
request world1 "etudiant_de(A, A)";;

This is false.


- : unit = ()


In [52]:
request world1 "apprend(A, A)";;

This is false.


- : unit = ()


In [53]:
request world1 "enseigne(A, A)";;

This is false.


- : unit = ()


In [54]:
request world1 "enseigne(alice, physique)";;

This is True.


- : unit = ()


In [28]:
request world1 "enseigne(alice, mathematiques)";; 

- : unit = ()


## Le métro londonien

In [39]:
let world =
let program = "
connected(bond_street,oxford_circus,central).
connected(oxford_circus,tottenham_court_road,central).
connected(bond_street,green_park,jubilee).
connected(green_park,charing_cross,jubilee).
connected(green_park,piccadilly_circus,piccadilly).
connected(piccadilly_circus,leicester_square,piccadilly).
connected(green_park,oxford_circus,victoria).
connected(oxford_circus,piccadilly_circus,bakerloo).
connected(piccadilly_circus,charing_cross,bakerloo).
connected(tottenham_court_road,leicester_square,northern).
connected(leicester_square,charing_cross,northern).

nearby(X,Y):-connected(X,Y,L).
nearby(X,Y):-connected(X,Z,L),connected(Z,Y,L).

reachable(X,Y):-connected(X,Y,L).
reachable(X,Y):-connected(X,Z,L),reachable(Z,Y).

path(X,Y,noroute):-connected(X,Y,L).
path(X,Y,route(Z,R)):-connected(X,Z,L),path(Z,Y,R)."
in read_program program;; (*connected(X,Y,L):-connected(Y,X,L).*)

val world : clause list =
  [Clause
    (Predicate ("connected",
      [Predicate ("bond_street", []); Predicate ("oxford_circus", []);
       Predicate ("central", [])]),
    []);
   Clause
    (Predicate ("connected",
      [Predicate ("oxford_circus", []);
       Predicate ("tottenham_court_road", []); Predicate ("central", [])]),
    []);
   Clause
    (Predicate ("connected",
      [Predicate ("bond_street", []); Predicate ("green_park", []);
       Predicate ("jubilee", [])]),
    []);
   Clause
    (Predicate ("connected",
      [Predicate ("green_park", []); Predicate ("charing_cross", []);
       Predicate ("jubilee", [])]),
    []);
   Clause
    (Predicate ("connected",
      [Predicate ("green_park", []); Predicate ("piccadilly_circus", []);
       Predicate ("piccadilly", [])]),
    []);
   Clause
    (Predicate ("connected",
      [Predicate ("piccadilly_circus", []);
       Predicate ("leicester_square", []); Predicate ("piccadilly", [])]),
    []);
   Clause
    (Predic

In [40]:
request world "connected(piccadilly_circus,leicester_square,piccadilly)";;

This is True.


- : unit = ()


In [41]:
request world "nearby(oxford_circus, charing_cross)";;

This is True.


- : unit = ()


In [42]:
request world "nearby(tottenham_court_road,W)";;

There is : W = leicester_square
There is : W = charing_cross


- : unit = ()


In [43]:
request world "reachable(bond_street, leicester_square)";;

This is True.
This is True.
This is True.
This is True.
This is True.


- : unit = ()


In [44]:
request world "connected(oxford_circus, bond_street, L)";;

This is false.


- : unit = ()


In [45]:
request world "path(oxford_circus,charing_cross,R)";;

There is : R = route(tottenham_court_road, route(leicester_square, noroute))
There is : R = route(piccadilly_circus, noroute)
There is : R = route(piccadilly_circus, route(leicester_square, noroute))


- : unit = ()
